In [ ]:
from transformers import BertModel, BertTokenizer

In [ ]:
# Import the required libraries
''' Use the AutoTokenizer.from_pretrained and AutoModel.from_pretrained methods to load the ProteinBERT model and the corresponding tokenizer from Hugging Face's model repository.
 These two functions can automatically select and download the pre-trained model based on the given model name.'''
from Bio import SeqIO
from transformers import BertModel, BertTokenizer
import torch
import numpy as np
import re
# Loading the ProtBERT model
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = BertModel.from_pretrained("Rostlab/prot_bert")
def encode_sequence(sequence):
    """Encode a single protein sequence, the tokenizer converts the sequence into a format acceptable to the model, calls the ProteinBERT model to encode the sequence, and finally returns the embedding vector."""
    encoded = tokenizer(sequence, return_tensors='pt')
    output = model(**encoded_input)
    with torch.no_grad():
        embedding = model(**encoded)['last_hidden_state']
    return embedding

def process_fasta(file_path, output_dir):
    embeddings = {}
    with open(file_path, 'r') as fasta_file:
        for record in SeqIO.parse(fasta_file, 'fasta'):
            sequence = str(record.seq)
            sequence = " ".join(sequence)
            embedding = encode_sequence(sequence)
            embeddings[record.id] = embedding.cpu().numpy()  # Convert the embedding to a NumPy array and save it

            # Print information to confirm the process
            print(f"Processed Sequence ID: {record.id}, Embedding Shape: {embedding.shape}")

    # Save the embedding vector
    for seq_id, emb in embeddings.items():
        np.save(f"{output_dir}/{seq_id}.npy", emb)

# Specifying the Output Directory
output_dir = '/content/drive/MyDrive/new_embeddings'
!mkdir -p {output_dir}

# Call the function to process the FASTA file and save the embedding vector
fasta_file_path = '/content/drive/MyDrive/newpractice2.fasta'
process_fasta(fasta_file_path, output_dir)

In [ ]:
import numpy as np

file_path = '/content/drive/MyDrive/new_embeddings/AP02484|1|training.npy'

data = np.load(file_path)

print(f"Data shape: {data.shape}")
print(f"Mean: {np.mean(data)}")
print(f"Standard deviation: {np.std(data)}")
print(f"First few entries:\n{data[:5]}")


Data shape: (1, 23, 1024)
Mean: 0.0030417400412261486
Standard deviation: 0.17069560289382935
First few entries:
[[[ 0.04684108  0.08342288  0.05901008 ... -0.07557765 -0.0041162
   -0.05276055]
  [ 0.10401953 -0.01991915  0.08518034 ... -0.00576243 -0.06395058
   -0.01161812]
  [-0.02504103 -0.01340439 -0.07927586 ... -0.01068252  0.05821817
    0.02694931]
  ...
  [ 0.03788668 -0.00519091 -0.12194559 ... -0.09377606  0.05076564
    0.05406712]
  [-0.07434283 -0.08396953 -0.06405934 ... -0.07258214  0.14850846
    0.10699935]
  [-0.06046145 -0.00445427 -0.0329536  ... -0.02252337 -0.02030955
    0.0483184 ]]]
